## Introduction

IPKISS is a python scripted framework for the design of integrated photonic components and circuits. If you are interested in learning more about the philosophy of IPKISS, please visit 
[IPKISS intro and history](http://docs.lucedaphotonics.com/3.0.1/tutorials/introduction/index.html).

In this training we will introduce you to the basic functionality in a interactive way using examples and exercices. We start with directly diving into the core architecture of IPKISS: PCells and views.


### PCells and views

IPKISS organizes these different aspects of the design flows using **PCells** and **Views**. 

<img src="_images/pcell_views.png" width=450>

In IPKISS each component or circuit is a PCell. This PCell contains all the  information about the component or the circuit. This information can cover aspects that are relevant at different stages of the design flow. This information is categorized into one or several views that are tightly linked to a particular stage in the design flow. For instance a PCell can have a **LayoutView** that describes how to build a GDSII description of the component/circuit. At the same time this PCell could also contain a **CapheModelView** which would contain a circuit model for the same component. Both PCells and Views are parametric through the use of properties. 

**PCells**, **Views** and **Properties** form the **backbone of the IPKISS framework** and learning how to use those is the purpose of this notebook.

Note: a more detailed introduction to pcells, views and properties can be found in the Luceda documentation, under "Guides" --> "Properties", and "Guides" --> "Parametric Cells". To open the documentation, open the "Luceda Control Center", and press the "Documentation" button. This will open the documentation in a browser window. On the left, you'll see a section "Guides".

### Let's Go: let's make a first PCell!

With this page, you have access to an interactive Python session in which you will get to know ipkiss and learn about exciting topics in photonics. This web application is based on the popular [IPython project](http://ipython.org/). It allows you to run python code from your webbrowser and explore the results.

Using these **notebooks** is very simple:

1. Click on the cell to select it.
2. Press `SHIFT+ENTER` on your keyboard or press the play button (<button class='fa fa-play icon-play btn btn-xs btn-default'></button>) in the toolbar above.

Try it on the cell below:

In [ ]:
# some python code to calculate pi:

pi_estimate = 0.0
terms_in_series = 500000

for k in range(terms_in_series):
    pi_estimate = pi_estimate + 4.0 * (-1)**k / (2 * k + 1.0)

print("Pi is approximately {0}".format(pi_estimate))

We will now create our first PCell. For now the cell won't have any views but it will illustrate how properties work.

#### Importing IPKISS and technologies

Before we start doing anything with IPKISS, we have to import the Ipkiss module into python, and a TECHNOLOGY file. The TECHNOLOGY file contains a series of constants that describe the process you will use to fabricate your chip. Here we use the standard ``silicon_photonics`` technology, shipped with Ipkiss itself.

In [ ]:
from technologies import silicon_photonics # Import the standard technology
from ipkiss3 import all as i3

In [ ]:
# Fix paths so that relative imports and plotting work in ipython 
import os, sys
sys.path.append('.')
%pylab inline
pylab.rcParams['figure.figsize'] = (12, 6)

#### A first PCell with properties

Now we are ready to start building a first PCell class which describes a Disk. Our disk will be described by two parameters:

- the disk radius
- the width of the Disk access waveguide.

<img src="_images/disk_annotated.png" width=450>

In [ ]:
class Disk(i3.PCell): #The name of the class is Disk. It inherits from PCell
    """
    Disk with a disk_radius, a disk_width and a coupler_spacing.
    """
    # Properties that describe the disl.
    disk_radius = i3.PositiveNumberProperty(doc="radius of the disk in micrometers")
    disk_wg_width = i3.PositiveNumberProperty(doc="width of the disk access waveguide in micrometers")
    coupler_spacing = i3.PositiveNumberProperty(doc="spacing between the waveguide and the disk")


Here we have:

- ``Disk`` : class name of the Disk,
- ``i3.PCell`` : class of a PCell. Using this syntax Disk inherits all the code that describes a PCell,
- ``disk_radius`` : Property describing the disk_radius. This is a positive number,
- ``disk_wg_width`` : Property describing the with of the the disk,
- ``coupler_spacing`` : Property describing the spacing beween the coupler waveguide and the disk.

We  have just created a Disk class with three properties. We now create a **Disk object** using the description provided by the class. When creating this object, we have to **assign values to the properties**.

In [ ]:
my_disk_object = Disk(disk_radius=10.0, disk_wg_width=1.0, coupler_spacing=0.5)
print my_disk_object.disk_radius
print my_disk_object.disk_wg_width
print my_disk_object.coupler_spacing

Through the statement ``my_disk_object = Disk(disk_radius=10.0, disk_wg_width=1.0, coupler_spacing=0.5)`` we created a disk object. We assigned the value 10.0 to ``disk_radius``, the value 1.0 to ``disk_wg_width``, and the value ``coupler_spacing`` to 0.5.

#### Exercise 1:

<font color="red"> Exercise 1: In the cell below: make your own PCell. In what follows try to make a PCell called **Square** with two properties:
- length
- width 

Then create an object of this square PCell called **my_square** and assign the value of 10.0 to the length and 5.0 to the width.


<font color="black">
Solution: [exercise_1.py](/edit/base_pcell_views_properties/support_files/pcell_properties/exercise_1.py)

In [ ]:
### Define Square,  your PCell class here.


In [ ]:
### Create my_square, an object of your pcell here


In [ ]:
# To check if your answer is correct execute this
from support_files.pcell_properties.checks import check_1
check_1(my_square)

#### Default values of properties

In many cases it is useful to assign default values to properties. That way, when you create an object of the PCell and you do not specify a property, the default value of the property will be used. This can be achieved in 2 ways:

* Directly from the property definition line, using the argument ``default=<something>``.
* Through a default method, called ``_default_<name_of_the_property>``. The function returns the default value.

In [ ]:
class DiskWithDefaults(i3.PCell): #The name of the class is DiskWithDefaults. It inherits from PCell
    """
    Disk with a disk_radius and a disk waveguide width. 
    """
    # Properties that describe the disk.
    disk_radius = i3.PositiveNumberProperty(default=10.0, doc="radius of the disk in mircometers")
    disk_wg_width = i3.PositiveNumberProperty(doc="width of the disk waveguide in micrometers")
    
    def _default_disk_wg_width(self):
        return self.disk_radius / 10.0


DiskWithDefaults is the same a Disk but
- when ``disk_radius`` is not specified, the value of 10.0 will be used.
- when ``disk_wg_width`` is not specified, the value of ``disk_wg_width/10`` will be used. 

Note that through the use of a default method, values of other properties can be used for the calculation. This is a very **powerful** concept for defining complex cells.

In [ ]:
# We can now make a disk cell without specifying any properties
my_disk2 = DiskWithDefaults()
print my_disk2.disk_radius
print my_disk2.disk_wg_width

In [ ]:
# But when we specify one of the properties the default value is not used.
my_disk3 = DiskWithDefaults(disk_wg_width=5.0)
print my_disk3.disk_radius
print my_disk3.disk_wg_width

#### Exercise 2:

<font color="red"> Exercise 2: In the cell below: make your own PCell. In what follows try to make a PCell called **SquareWithDefaults** with two properties:
- length with default value 5.0
- width with a default equal to the length



<font color="black">
Solution: [exercise_2.py](/edit/base_pcell_views_properties/support_files/pcell_properties/exercise_2.py)

In [ ]:
### Create your PCell class here.


In [ ]:
from support_files.pcell_properties.checks import check_2
check_2(SquareWithDefaults)

Congratulations! You have now been introduced to the basic framework of PCells. In the next step we will show you how to add a layout view to the disk.

#### Up Next:




<div>
<div style="width: 20%; display: inline; margin:0; float: left"> <p> 
<a href="04_circuitmodelview.ipynb"> <img src="_images/smatrix.png" width=400 ></a> </p> </div>
<div style="width: 49%;  display: inline; float: right; margin:0"> <p> <a href="04_circuitmodelview"> Notebook: Circuit Model</a> </p> </div>
</div>